In [1]:
import os
import glob
import pandas as pd
os.chdir("/content/drive/MyDrive/csv")

In [2]:
extension = 'csv'
all_filenames = [i for i in glob.glob('*.{}'.format(extension))]

In [3]:
# combine all files in the list
combined_csv = pd.concat([pd.read_csv(f) for f in all_filenames ])
# export to csv
combined_csv.to_csv("combined_csv.csv", index=False)

In [4]:
import pandas as pd
import numpy as np


In [5]:
combined_csv = pd.read_csv('/content/drive/MyDrive/csv/combined_csv.csv')
print(combined_csv.shape)
combined_csv.head()

(186392, 17)


,Unnamed: 0,City,State,Population,Violent crime,Murder and nonnegligent manslaughter,Rape2,Robbery,Aggravated assault,Property crime,Burglary,Larceny- theft,Motor vehicle theft,Arson,Rape1,Arson3,Rape
0,4.0,Hoover,AL,85670.0,114.0,4.0,15.0,27.0,68.0,1922.0,128.0,1694.0,100.0,2.0,NaN,NaN,NaN
1,4.0,Anchorage,AK,287731.0,3581.0,32.0,NaN,621.0,2388.0,12261.0,1692.0,9038.0,1531.0,93.0,540.0,NaN,NaN
2,5.0,Bethel,AK,6544.0,130.0,1.0,NaN,3.0,79.0,132.0,20.0,84.0,28.0,12.0,47.0,NaN,NaN
3,6.0,Bristol Bay Borough,AK,852.0,2.0,0.0,NaN,0.0,2.0,20.0,5.0,8.0,7.0,0.0,0.0,NaN,NaN
4,7.0,Cordova,AK,2150.0,0.0,0.0,NaN,0.0,0.0,7.0,1.0,6.0,0.0,0.0,0.0,NaN,NaN


In [6]:
combined_csv = combined_csv.fillna(0)

In [7]:
combined_csv.reset_index(drop=True).head()

,Unnamed: 0,City,State,Population,Violent crime,Murder and nonnegligent manslaughter,Rape2,Robbery,Aggravated assault,Property crime,Burglary,Larceny- theft,Motor vehicle theft,Arson,Rape1,Arson3,Rape
0,4.0,Hoover,AL,85670.0,114.0,4.0,15.0,27.0,68.0,1922.0,128.0,1694.0,100.0,2.0,0.0,0.0,0.0
1,4.0,Anchorage,AK,287731.0,3581.0,32.0,0.0,621.0,2388.0,12261.0,1692.0,9038.0,1531.0,93.0,540.0,0.0,0.0
2,5.0,Bethel,AK,6544.0,130.0,1.0,0.0,3.0,79.0,132.0,20.0,84.0,28.0,12.0,47.0,0.0,0.0
3,6.0,Bristol Bay Borough,AK,852.0,2.0,0.0,0.0,0.0,2.0,20.0,5.0,8.0,7.0,0.0,0.0,0.0,0.0
4,7.0,Cordova,AK,2150.0,0.0,0.0,0.0,0.0,0.0,7.0,1.0,6.0,0.0,0.0,0.0,0.0,0.0


In [8]:
combined_csv['Arson'] = combined_csv.apply(lambda x: x['Arson'] +  x['Arson3'], axis=1)

In [9]:
combined_csv['Rape'] = combined_csv.apply(lambda x: x['Rape2'] +  x['Rape1'], axis=1)

In [10]:
combined_csv = combined_csv.drop(columns=['Unnamed: 0', 'Arson3', 'Rape2', 'Rape1'])

In [11]:
combined_csv.head()

,City,State,Population,Violent crime,Murder and nonnegligent manslaughter,Robbery,Aggravated assault,Property crime,Burglary,Larceny- theft,Motor vehicle theft,Arson,Rape
0,Hoover,AL,85670.0,114.0,4.0,27.0,68.0,1922.0,128.0,1694.0,100.0,2.0,15.0
1,Anchorage,AK,287731.0,3581.0,32.0,621.0,2388.0,12261.0,1692.0,9038.0,1531.0,93.0,540.0
2,Bethel,AK,6544.0,130.0,1.0,3.0,79.0,132.0,20.0,84.0,28.0,12.0,47.0
3,Bristol Bay Borough,AK,852.0,2.0,0.0,0.0,2.0,20.0,5.0,8.0,7.0,0.0,0.0
4,Cordova,AK,2150.0,0.0,0.0,0.0,0.0,7.0,1.0,6.0,0.0,0.0,0.0


In [12]:
combined_csv.to_csv("fbi_crime_uscities.csv", index=False)
